# Toegang tot de database

> tags: []

# Tutorial: Accessing the database from a Vaadin Flow view

Learn how to connect a Vaadin Flow view to the backend through Spring Services.


We hebben de documentatie gereorganiseerd om u een meer gestroomlijnde navigatiestructuur te bieden

1.  [Documentatie](https://vaadin.com/docs/latest/)
2.  [Tutorial](https://vaadin.com/docs/latest/tutorial)
3.  Toegang tot de database

-   [Een introductie tot Spring Boot](https://vaadin.com/docs/latest/tutorial/database-access/#an-introduction-to-spring-boot)
-   [Backend Overzicht](https://vaadin.com/docs/latest/tutorial/database-access/#backend-overview)
-   [Een service maken voor toegang tot de database](https://vaadin.com/docs/latest/tutorial/database-access/#creating-a-service-for-accessing-the-database)
-   [Filtering implementeren in de repository](https://vaadin.com/docs/latest/tutorial/database-access/#implementing-filtering-in-the-repository)
-   [De back-endservice gebruiken vanuit de weergave](https://vaadin.com/docs/latest/tutorial/database-access/#using-the-backend-service-from-the-view)

In het vorige hoofdstuk hebt u de weergave gemaakt met vaadin-componenten en -indelingen. In dit hoofdstuk verbindt u de weergave met de backend om gegevens weer te geven en bij te werken.

U kunt de backend-code vinden in de directory.`src/main/java`

Dit hoofdstuk heeft betrekking op:

-   Lente Laars.
    
-   Hoe maak je een Spring Service voor interfacing van de backend.
    
-   Toegang krijgen tot een service vanuit de weergave.
    

## [](https://vaadin.com/docs/latest/tutorial/database-access/#an-introduction-to-spring-boot)Een introductie tot Spring Boot

Vaadin gebruikt [Spring Boot](https://spring.io/projects/spring-boot) op de server. Spring Boot is een eigenwijze, conventie-over-configuratie benadering voor het maken van Spring-applicaties. Spring Boot automatiseert veel van de vereiste configuratie en beheert een ingebouwde Tomcat-server, zodat u de toepassing niet op een aparte server hoeft te implementeren.

In deze zelfstudie gebruikt u de volgende functies die zijn geconfigureerd door Spring Boot:

-   [Spring Data](https://spring.io/projects/spring-data) voor toegang tot de database via [JPA](https://spring.io/projects/spring-data-jpa) en Hibernate.
    
-   Een geïntegreerde [H2-database](https://www.h2database.com/html/main.html) voor ontwikkeling en [PostgreSQL](https://www.postgresql.org/) voor productie.
    
-   Spring Boot DevTools voor het automatisch herladen van code.
    
-   Geïntegreerde Tomcat-server voor implementatie.
    
-   [Spring Security](https://spring.io/projects/spring-security) voor het verifiëren van gebruikers.
    

## [](https://vaadin.com/docs/latest/tutorial/database-access/#backend-overview)Backend Overzicht

De starter die u hebt gedownload, bevat de _entiteiten_ en _opslagplaatsen die_ u nodig hebt, samen met een voorbeeldgegevensgenerator.

### [](https://vaadin.com/docs/latest/tutorial/database-access/#domain-model-entities)Domeinmodel: entiteiten

De Vaadin CRM-applicatie heeft drie JPA-entiteiten die deel uitmaken van het domeinmodel: , , en . Een contactpersoon hoort bij een bedrijf en heeft een status.`Contact``Company``Status`

U kunt de entiteiten in het pakket vinden.`com.example.application.data.entity`

![Diagram met entiteitsmodel](https://vaadin.com/docs/latest/static/27d217d5a311ce181dc3acdd63b507ca/03979/entity-model.png)

### [](https://vaadin.com/docs/latest/tutorial/database-access/#database-access-repositories)Databasetoegang: opslagplaatsen

The application uses Spring Data JPA repositories for database access. Spring Data provides implementations of basic create, read, update, and delete (CRUD) database operations when you extend from the interface.`JpaRepository`

You can find the repositories in the package.`com.example.application.data.service`

### [](https://vaadin.com/docs/latest/tutorial/database-access/#sample-data-generator)Sample Data Generator

The package contains a sample data generator that populates the database with data. It uses , which Spring Boot runs when the application starts up.`com.example.application.data.generator``CommandLineRunner`

## [](https://vaadin.com/docs/latest/tutorial/database-access/#creating-a-service-for-accessing-the-database)Creating a Service for Accessing the Database

Instead of accessing the database directly from the view, you will create a Spring Service. The service class handles the application’s business logic and, in larger applications, it will often transform database entities into data-transfer objects (DTO) for views. In this tutorial, you will create a single service that provides all the methods you need.

Create a new class, , in the package with the following content:`CrmService.java``data.service`

```
package com.example.application.data.service;

import com.example.application.data.entity.Company;
import com.example.application.data.entity.Contact;
import com.example.application.data.entity.Status;
import com.example.application.data.repository.CompanyRepository;
import com.example.application.data.repository.ContactRepository;
import com.example.application.data.repository.StatusRepository;
import org.springframework.stereotype.Service;

import java.util.List;

@Service 1
public class CrmService {

    private final ContactRepository contactRepository;
    private final CompanyRepository companyRepository;
    private final StatusRepository statusRepository;

    public CrmService(ContactRepository contactRepository,
                      CompanyRepository companyRepository,
                      StatusRepository statusRepository) { 2
        this.contactRepository = contactRepository;
        this.companyRepository = companyRepository;
        this.statusRepository = statusRepository;
    }

    public List<Contact> findAllContacts(String stringFilter) {
        if (stringFilter == null || stringFilter.isEmpty()) { 3
            return contactRepository.findAll();
        } else {
            return contactRepository.search(stringFilter);
        }
    }

    public long countContacts() {
        return contactRepository.count();
    }

    public void deleteContact(Contact contact) {
        contactRepository.delete(contact);
    }

    public void saveContact(Contact contact) {
        if (contact == null) { 4
            System.err.println("Contact is null. Are you sure you have connected your form to the application?");
            return;
        }
        contactRepository.save(contact);
    }

    public List<Company> findAllCompanies() {
        return companyRepository.findAll();
    }

    public List<Status> findAllStatuses(){
        return statusRepository.findAll();
    }
}
```

1.  The annotation will make this a Spring-managed service that you can inject into your view.`@Service`
    
2.  Use Spring constructor injection to autowire the database repositories.
    
3.  Check if there is an active filter, return either all contacts or use the repository to filter based on the string.
    
4.  Service classes often include validation and other business rules before persisting data. Here, you check that you aren’t accidentally trying to save a object.`null`
    

## [](https://vaadin.com/docs/latest/tutorial/database-access/#implementing-filtering-in-the-repository)Implementing Filtering in the Repository

Add the method to the contacts repository to provide the service class with the required method for filtering contacts.`search()`

```
public interface ContactRepository extends JpaRepository<Contact, Long> {

  @Query("select c from Contact c " +
      "where lower(c.firstName) like lower(concat('%', :searchTerm, '%')) " +
      "or lower(c.lastName) like lower(concat('%', :searchTerm, '%'))") 1
    List<Contact> search(@Param("searchTerm") String searchTerm); 2
}
```

1.  Uses the annotation to define a custom query. In this case, it checks if the string matches the first or the last name, and ignores the case. The query uses [Java Persistence Query Language (JPQL)](https://en.wikipedia.org/wiki/Java_Persistence_Query_Language) which is an SQL-like language for querying JPA-managed databases.`@Query`
    

Note that you do not need to implement the method. Spring Data will provide the implementation based on the query.

## [](https://vaadin.com/docs/latest/tutorial/database-access/#using-the-backend-service-from-the-view)Using the Backend Service From the View

You can now inject the into the list view to access the backend.`CrmService`

```
package com.example.application.views.list;

import com.example.application.data.entity.Contact;
import com.example.application.data.service.CrmService;
import com.vaadin.flow.component.Component;
import com.vaadin.flow.component.button.Button;
import com.vaadin.flow.component.grid.Grid;
import com.vaadin.flow.component.orderedlayout.HorizontalLayout;
import com.vaadin.flow.component.orderedlayout.VerticalLayout;
import com.vaadin.flow.component.textfield.TextField;
import com.vaadin.flow.data.value.ValueChangeMode;
import com.vaadin.flow.router.PageTitle;
import com.vaadin.flow.router.Route;

import java.util.Collections;

@Route(value = "")
@PageTitle("Contacts | Vaadin CRM")
public class ListView extends VerticalLayout {
    Grid<Contact> grid = new Grid<>(Contact.class);
    TextField filterText = new TextField();
    ContactForm form;
    CrmService service;

    public ListView(CrmService service) { 1
        this.service = service;
        addClassName("list-view");
        setSizeFull();
        configureGrid();
        configureForm();

        add(getToolbar(), getContent());
        updateList(); 2
    }

    private Component getContent() {
        HorizontalLayout content = new HorizontalLayout(grid, form);
        content.setFlexGrow(2, grid);
        content.setFlexGrow(1, form);
        content.addClassNames("content");
        content.setSizeFull();
        return content;
    }

    private void configureForm() {
        form = new ContactForm(service.findAllCompanies(), service.findAllStatuses()); 3
        form.setWidth("25em");
    }

    private void configureGrid() {
        grid.addClassNames("contact-grid");
        grid.setSizeFull();
        grid.setColumns("firstName", "lastName", "email");
        grid.addColumn(contact -> contact.getStatus().getName()).setHeader("Status");
        grid.addColumn(contact -> contact.getCompany().getName()).setHeader("Company");
        grid.getColumns().forEach(col -> col.setAutoWidth(true));
    }

    private HorizontalLayout getToolbar() {
        filterText.setPlaceholder("Filter by name...");
        filterText.setClearButtonVisible(true);
        filterText.setValueChangeMode(ValueChangeMode.LAZY);
        filterText.addValueChangeListener(e -> updateList()); 4

        Button addContactButton = new Button("Add contact");

        HorizontalLayout toolbar = new HorizontalLayout(filterText, addContactButton);
        toolbar.addClassName("toolbar");
        return toolbar;
    }

    private void updateList() { 5
        grid.setItems(service.findAllContacts(filterText.getValue()));
    }
}
```

1.  Autowire through the constructor. Save it in a field, so you can access it from other methods.`CrmService`
    
2.  Call once you have constructed the view.`updateList()`
    
3.  Use the service to fetch companies and statuses.
    
4.  Call any time the filter changes.`updateList()`
    
5.  `updateList()` sets the grid items by calling the service with the value from the filter text field.
    

Build the project, refresh the browser, and verify that you can now see contacts in the grid. Try filtering the contents by typing into the filter text field.

![The grid displays 4 contacts after filtering with the string "ja"](https://vaadin.com/docs/latest/static/2da664cf62433db382f647c9e2368606/03979/contacts-grid-filtering.png)

1EA60808-40B7-4F0C-8B71-C0CB905299D2

[TutorialCreating a Component](https://vaadin.com/docs/latest/tutorial/creating-a-component)[TutorialForms and Validation](https://vaadin.com/docs/latest/tutorial/forms-and-validation)

**Was this page helpful?**  
Leave a comment or a question below. You can also join the [chat on Discord](https://discord.gg/MYFq5RTbBn) or [ask questions on StackOverflow](https://stackoverflow.com/questions/tagged/vaadin).
